# importando tabula e arquivos pdf
- importante que os PDFs estejam todos na mesma página dos arquivos

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import tabula
import pandas as pd

lista_suap = tabula.read_pdf("suap/2022_07_RMA_suap.pdf", pages="1", encoding='Ansi')
mes = '2022-07'
# lista em dataframe
df_suap = lista_suap[0]

# removendo linhas com valores nulos
df_suap = df_suap.dropna()
display(df_suap)

df_suap = df_suap[:-1]

# removendo e renomeando colunas
df_suap = df_suap.drop(columns=['Nome', 'Estoque', 'Entrada'])
df_suap['ED'] = df_suap['Cod.']
df_suap['saida'] = df_suap['Saída']
df_suap['saldo_suap'] = df_suap['Estoque.1']
df_suap = df_suap.drop(columns=['Cod.', 'Estoque.1', 'Saída'])

# substituindo caracteres para a conversão em numero
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(".", "", regex=True)
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(",", ".", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(".", "", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(",", ".", regex=True)


df_suap['saldo_suap'] = pd.to_numeric(df_suap['saldo_suap'])
df_suap['saida'] = pd.to_numeric(df_suap['saida'])
print("{:.2f}".format(df_suap['saldo_suap'].sum()))
print(df_suap.info())
display(df_suap)

,Cod.,Nome,Estoque,Entrada,Saída,Estoque.1
1,14,MATERIAL EDUCATIVO E ESPORTIVO,"40.583,23","0,00","0,00","40.583,23"
2,16,MATERIAL DE EXPEDIENTE,"63.274,23","0,00","0,00","63.274,23"
3,17,MATERIAL DE PROCESSAMENTO DE DADOS,"16.356,19","0,00","0,00","16.356,19"
4,19,MATERIAL DE ACONDICIONAMENTO E,"2.332,87","0,00","0,00","2.332,87"
6,21,MATERIAL DE COPA E COZINHA,"7.290,60","0,00","0,00","7.290,60"
7,22,MATERIAL DE LIMPEZA E PROD. DE,"29.104,99","0,00","0,00","29.104,99"
9,23,"UNIFORMES, TECIDOS E AVIAMENTOS","16.512,14","0,00","0,00","16.512,14"
10,24,MATERIAL P/ MANUT.DE BENS,"43.880,29","0,00","0,00","43.880,29"
12,25,MATERIAL P/ MANUTENCAO DE BENS,"226,96","0,00","0,00","226,96"
14,26,MATERIAL ELETRICO E ELETRONICO,"29.094,22","0,00","0,00","29.094,22"


285878.04
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 21
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ED          16 non-null     object 
 1   saida       16 non-null     float64
 2   saldo_suap  16 non-null     float64
dtypes: float64(2), object(1)
memory usage: 512.0+ bytes
None


,ED,saida,saldo_suap
1,14,0.0,40583.23
2,16,0.0,63274.23
3,17,0.0,16356.19
4,19,0.0,2332.87
6,21,0.0,7290.60
7,22,0.0,29104.99
9,23,0.0,16512.14
10,24,0.0,43880.29
12,25,0.0,226.96
14,26,0.0,29094.22


In [2]:
# parâmetros lattice=True e guess=False para conseguir reconhecer e montar tabelas em arquivos com formatação diferente

lista_siafi = tabula.read_pdf("siafi/2022_07_RMA_siafi.pdf", guess=False, pages="1-2")

tabela1 = lista_siafi[0]
tabela1.columns = tabela1.iloc[7]
tabela1 = tabela1[8:20]

tabela2 = lista_siafi[1]
tabela2.columns = tabela2.iloc[7]
tabela2 = tabela2[8:15]

# Renomeando coluna por causa do caracter especial $

tabela1.columns = ['conta']
tabela2.columns = ['conta']

# unindo tabelas pela coluna conta
df_siafi = pd.merge(tabela1, tabela2, how = 'outer', on = 'conta')

# fatiando coluna para obter colunas ED e Saldo
df_siafi['ED'] = df_siafi['conta'].str[:4]
df_siafi['saldo_siafi'] = df_siafi['conta'].str[4:-1]

# removendo coluna ajustada
df_siafi = df_siafi.drop(columns=['conta'])

# substituindo caracteres para a conversão em numero
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(".", "", regex=True)
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(",", ".", regex=True)
df_siafi['ED'] = df_siafi['ED'].str.replace("P ", "", regex=True)
df_siafi = df_siafi.drop(index=[17,18])

df_siafi = df_siafi.drop(index=16)
df_siafi['saldo_siafi'] = pd.to_numeric(df_siafi['saldo_siafi'])

print("{:.2f}".format(df_siafi['saldo_siafi'].sum()))
display(df_siafi)
print(df_siafi.info())
# display(df_siafi)

286089.09


,ED,saldo_siafi
0,14,40583.23
1,16,63274.23
2,17,16356.19
3,19,2332.87
4,21,7290.60
5,22,28456.79
6,23,16512.14
7,24,44507.29
8,25,226.96
9,26,29326.47


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ED           16 non-null     object 
 1   saldo_siafi  16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 384.0+ bytes
None


In [3]:
# unindo data frames do suap e siafi em uma unica tabela
df_rma = pd.merge(df_suap, df_siafi, how = 'inner', on='ED')
df_rma = df_rma.reindex(columns=['ED','saldo_suap', 'saida', 'saldo_siafi'])

# calculando a diferença entre colunas 
df_rma['diferenca'] = df_rma['saldo_suap'] - df_rma['saldo_siafi']

# definindo a ação para cafa resultado
acao = []
for valor in df_rma['diferenca']:
    
    if valor > 0:
        baixa = "baixa no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "baixa no siafi"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

df_rma['acao'] = acao

display(df_rma)
df_rma.to_clipboard()
df_rma.to_excel(f'saida/{mes}.xlsx')

,ED,saldo_suap,saida,saldo_siafi,diferenca,acao
0,14,40583.23,0.0,40583.23,0.00,adequado
1,16,63274.23,0.0,63274.23,0.00,adequado
2,17,16356.19,0.0,16356.19,0.00,adequado
3,19,2332.87,0.0,2332.87,0.00,adequado
4,21,7290.60,0.0,7290.60,0.00,adequado
5,22,29104.99,0.0,28456.79,648.20,baixa no suap
6,23,16512.14,0.0,16512.14,0.00,adequado
7,24,43880.29,0.0,44507.29,-627.00,baixa no siafi
8,25,226.96,0.0,226.96,0.00,adequado
9,26,29094.22,0.0,29326.47,-232.25,baixa no siafi
